In [5]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import math
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import *


In [4]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [20]:
"""
    cross validation
"""

datapath ='/home02/chenhuangrong/RFH_feature.csv'
datapath2 ='/home02/chenhuangrong/KNN.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
train_data=train_data.values
train_data2=pd.read_csv(datapath2,header=None,index_col=False)
train_data2=train_data2.values
train_data=np.concatenate((train_data,train_data2),axis=1)
for select_num in xrange(1,204):
    X = np.array(train_data)
    F,pval=f_classif(X,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=X[:,select_list]
    Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
    Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
    Y.extend(Y2)
    Y = np.array(Y)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
#     pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/KNN_RFH.csv',header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
    print X.shape[1]
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/KNN_RFH.xls')
    # y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
    # Y=pd.DataFrame(Y)    
    # y_predict_proba=pd.DataFrame(y_predict_proba)
    # y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
    # pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
    # y_predict=pd.DataFrame(y_predict)
    # y_predict_=pd.concat([Y,y_predict],axis=1)
    # pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

/usr/local/lib/python2.7/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  3  96  97  98 100 101 102 104 105 106] are constant.
  UserWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
1


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
2
0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
3
0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
4
0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
5
0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
6
0.5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
7
0.5


In [19]:
A=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
A[:,[0,1,2]]

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])